# Decision Trees


Decision trees are flowchart-like structures composed by nodes, branches and leaves. A node is a test on a certain attribute, a branch represents the outcome of a test and finally, a leaf corresponds to the decision taken.

This takes us to decision tree learning. The goal of this method is to create a model that predicts the outcome of a given variable based on several inputs.

A simple example as seen in Tom Mitchell's "Machine Learning" is the following:
### Example: Playing tennis
![alternate text](https://www.cise.ufl.edu/~ddd/cap6635/Fall-97/Short-papers/Image3.gif)

------------------------------------------------------------------------------------------------------------------
Pros:
==

* Simple to understand and to visualize.

* Requires little data preparation.

* Complexity of the algorithm is logarithmic in the number of points used to train the tree.

Cons:
==

* Prone to overfitting. Aside from choosing the right values for the right parameters, it might require extra techniques, such as pruning to avoid this problem.

* Decision trees might be unstable, with small variations in data resulting on very different trees.

More at: http://scikit-learn.org/stable/modules/tree.html#

------------------------------------------------------------------------------------------------------------------
Mathematical Background:
==

There are plenty of implementations of Decision Trees. Scikit implements an optimised version of the **CART (Classification and Regression Trees)** algorithm. This algorithm works by constructing binary trees (a tree data structure in which each node has at most two children) using the feature and threshold that allow the tree to gain the biggest amount of information at each node.



In this workshop we will use entropy as the measure of information gain and as the criterion in our decision tree classifier. Information gain is based on the concept of entropy from information theory.

Information gain may be defined as:

$$IG(T,a) = H(T) - H(T|a)$$

which can be translated as *"Information gain is given by the entropy of the parent node minus the weighted sum of entropy of the children nodes"*. 

It's also important to define entropy:

$$H(T) = -\sum_{i=1}^j p_i log_2 p_i$$

where $p_i$ represents the percentage of each class present in the child node that results in a split in the tree.

A good way to understand the concept of entropy is by looking at the following graphic:

![alternate text](https://upload.wikimedia.org/wikipedia/commons/thumb/2/22/Binary_entropy_plot.svg/320px-Binary_entropy_plot.svg.png?1490734279242)

As we can see, entropy is a measure that has values between 0 and 1. When in an event with two possible outcomes, one has probability 0 and the other one 1, we have absolute certainty about the outcome and therefore, zero entropy on that decision. When we have two events that are equally probable, the uncertainty on the outcome is maximum and so is the entropy.

Since the entropy of the parent node is always the same for a certain tree split, we may approach this problem by finding the decision that minimizes the sum of the entropy of the children nodes.



# Decision Tree implementation

In [1]:
# Data visualization
import pandas as pd

iris_df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/iris.csv")
print("Iris dataframe)",iris_df.head(),sep="\n")
print("\n....\n")
print(iris_df.tail(),sep="\n")
print("\n....\n")
print(iris_df.info())
print("\n....\n")
print(iris_df.describe())

Iris dataframe)
   SepalLength  SepalWidth  PetalLength  PetalWidth         Name
0          5.1         3.5          1.4         0.2  Iris-setosa
1          4.9         3.0          1.4         0.2  Iris-setosa
2          4.7         3.2          1.3         0.2  Iris-setosa
3          4.6         3.1          1.5         0.2  Iris-setosa
4          5.0         3.6          1.4         0.2  Iris-setosa

....

     SepalLength  SepalWidth  PetalLength  PetalWidth            Name
145          6.7         3.0          5.2         2.3  Iris-virginica
146          6.3         2.5          5.0         1.9  Iris-virginica
147          6.5         3.0          5.2         2.0  Iris-virginica
148          6.2         3.4          5.4         2.3  Iris-virginica
149          5.9         3.0          5.1         1.8  Iris-virginica

....

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
SepalLength    150 non-null float64
SepalWidth     150 n

------------------------------------------------------------------------------------------------------------------

As we can see, what we intend to use as label in our learning model (the names of the plants), is not a float and this will bring us problems later on when we try to train the model.. let's then start by encoding them to floats!

In [2]:
# Encoding the labels to be usabel in our training model
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(iris_df["Name"])
y = le.transform(iris_df["Name"])

print("Our column \"Name\" now looks like this:\n {}".format(y))

Our column "Name" now looks like this:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


------------------------------------------------------------------------------------------------------------------

Now that this problem is solved, we will split our dataset into two, one training set and on test set. This will allow us to check if our system is generalizing properly to unseen data and therefore, whether it is overfitting or not.

In [3]:
# Split the data into a training and test set
from sklearn.model_selection import train_test_split

X = iris_df.drop("Name",1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

------------------------------------------------------------------------------------------------------------------
We can now train our model!

In [4]:
# Training the model with an decision tree classifier
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=3, min_samples_split=5, 
							min_samples_leaf=6, min_weight_fraction_leaf=0.0, max_features=None, 
							random_state=None, max_leaf_nodes=None, min_impurity_split=1e-07, class_weight=None, 
							presort=False)
dt.fit(X_train,y_train)

# Predicted labels for our test features - not needed, it's just to visualize it
Y_pred = dt.predict(X_test)

print("The predicted values are: \n {}".format(Y_pred))
print("The values that should be obtained are: \n {}".format(y_test))

score_train = dt.score(X_train,y_train)
score_test = dt.score(X_test, y_test)

print("Our model has a score of {} for the train set and a score of {} for the test set".format(score_train,score_test))

The predicted values are: 
 [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0 1 1 2 1 2]
The values that should be obtained are: 
 [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0 1 2 2 1 2]
Our model has a score of 0.95 for the train set and a score of 0.98 for the test set


------------------------------------------------------------------------------------------------------------------
Python allows us to visualize random trees. But since it requires some extra non-common libraries we will only show examples of obtainable trees. We provide the code nonetheless, so you can try it at home!

In [5]:
# Visualize the decision tree

"""
import pydotplus 
from sklearn import tree

features = X.columns[:4]

dot_data = tree.export_graphviz(dt, out_file=None,
								feature_names = features, class_names = ['setosa','versicolor','virginica'],
								filled = True, rounded = True) 
graph = pydotplus.graph_from_dot_data(dot_data) 
graph.write_pdf("iris.pdf")"""

'\nimport pydotplus \nfrom sklearn import tree\n\nfeatures = X.columns[:4]\n\ndot_data = tree.export_graphviz(dt, out_file=None,\n\t\t\t\t\t\t\t\tfeature_names = features, class_names = [\'setosa\',\'versicolor\',\'virginica\'],\n\t\t\t\t\t\t\t\tfilled = True, rounded = True) \ngraph = pydotplus.graph_from_dot_data(dot_data) \ngraph.write_pdf("iris.pdf")'

------------------------------------------------------------------------------------------------------------------
Using a maximum depth of 1, minimum number of samples at a leaf node of 5, we obtain a **train score of 0.66 and a test score of 0.68**.

![alternate text](http://i65.tinypic.com/21mi1s8.png)

------------------------------------------------------------------------------------------------------------------
But we can do better. Using a maximum depth of 2, our **train score improves to 0.96 and our test score to 0.98**.

![alternate text](http://i63.tinypic.com/21jpspf.png)

------------------------------------------------------------------------------------------------------------------
What if we allow our tree to have a maximum depth of 3? We can finally get a **test score of 1, while having a train score of 0.98!**

![alternate text](http://i65.tinypic.com/2nm3wba.png)

------------------------------------------------------------------------------------------------------------------
Let's now keep this maximum depth, but increase the minimum number of samples at a leaf node to 6. **Our train score decreased to 0.95 and our test score to 0.98.** This means that incresing this value made our system worse at learning the data..

![alternate text](http://i68.tinypic.com/2qus01w.png)

------------------------------------------------------------------------------------------------------------------
And can we overfit such a small and simple dataset? Using a maximum depth of 6 and reducing our parameters min_samples_leaf and min_samples_split to 1 and 2, respectively, **we now have a train score of 1 and a test score of 0.98.** As we've seen in out last workshop, having a better accuracy in out training set than in out test set is a clear sign of overfitting!

![alternate text](http://i63.tinypic.com/2njvwi0.png)

And to visualize the space segmentation that this algorithm produces:

![alternate text](http://scikit-learn.org/stable/_images/sphx_glr_plot_iris_0011.png)

------------------------------------------------------------------------------------------------------------------
**Key tips on implementing decision trees**
==

* **Use the right ratio of samples to number of features** - Decision trees have the tendency to overfit when trained with data that uses a large number of features. It's a good idea to perform dimensionality reduction beforehand.

* **Start with maximum deepness of 3** - Start with this value and change it accordingly to your results. It's an important aspect to prevent overfitting.

* **Start with a value of 5 for the minimum number of samples on min_samples_split and min_samples_leaf** - This parameters are used to control the number of samples at a leaf node. As we've seen in our examples, lower values will make your tree overfit (you allow your tree to have leafs with a smaller number of samples -> tree will generalize poorly to unseen data), while higher values will prevent your tree to learn the data (if each leaf has too have too many samples -> your tree will be to general and will not describe your data).

more tips at: http://scikit-learn.org/stable/modules/tree.html#tips-on-practical-use